In [20]:
cd/content/drive/My Drive/투빅스 13기

/content/drive/My Drive/투빅스 13기


In [0]:
ls

 final_keras_submission.csv   relu_reaky_selu_submission.csv
 FireKeras.ipynb              relu_selu_adamax_submission.csv
 FirePytorch.ipynb            relu_selu_nadam_submission.csv
 kaggle.json                  relu_selu_submission.csv
 _modified_submission         sample_submission.csv
 modified_submission          sgd_submission
 modified_submission.csv      test_df.csv
'*.png'                       test_df.csv.zip
 radam_modified_submission    train_df.csv
 radam_submission             train_df.csv.zip
 radam_submission.csv         untitled


In [0]:
import pandas as pd
import numpy as np

In [0]:
sample_submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")

In [0]:
X = train.iloc[:,1:].values / 255
y = train.iloc[:,0].values
X_test = test.iloc[:,1:].values / 255

## Dataset 이랑 Loader 정의하기

In [0]:
from torch.utils.data import Dataset, DataLoader

class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

class TestDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        return X

In [0]:
traindataloader = DataLoader(TrainDataset(X, y), batch_size=256, shuffle=True, num_workers=4)
testdataloader = DataLoader(TestDataset(X_test, y=None), batch_size=4, shuffle=False, num_workers=4)

## Layer 별 설정 해주기  
https://pytorch.org/docs/stable/nn.html

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128,128)
        self.fc5 = nn.Linear(128, 10)
        self.elu = nn.ELU()
        self.lrelu = nn.LeakyReLU()
        self.prelu = nn.PReLU()
        self.dropout1 = torch.nn.Dropout(p=0.2)
        self.dropout2 = torch.nn.Dropout(p=0.2)
        self.dropout3 = torch.nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(F.relu(x))
        x = self.dropout1(x)
        x = self.fc3(self.elu(x))
        x = self.dropout2(x)
        x = self.fc4(self.lrelu(x)) # LeakyReLu만 사용해봄
        x = self.dropout3(x)
        x = self.fc5(x)
        return x

net = Net()

In [10]:
net

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (lrelu): LeakyReLU(negative_slope=0.01)
  (prelu): PReLU(num_parameters=1)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
  (dropout3): Dropout(p=0.2, inplace=False)
)

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=2e-3, weight_decay=1e-3)

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [13]:
net.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (lrelu): LeakyReLU(negative_slope=0.01)
  (prelu): PReLU(num_parameters=1)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
  (dropout3): Dropout(p=0.2, inplace=False)
)

In [14]:
num_epochs = 50
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(traindataloader):
        inputs, labels = data
        inputs = torch.tensor(inputs, device=device).float()
        labels = torch.tensor(labels, device=device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # print(type(inputs), type(labels), type(outputs))
        # forward + backward + optimize
        outputs = net(inputs)
        # print(type(inputs), type(labels), type(outputs))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    print("Epoch : {} loss: {}".format(epoch, running_loss))
print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Epoch : 0 loss: 50.16538452357054
Epoch : 1 loss: 16.20941462367773
Epoch : 2 loss: 12.610262982547283
Epoch : 3 loss: 10.33344815671444
Epoch : 4 loss: 9.746505178511143
Epoch : 5 loss: 8.494967066682875
Epoch : 6 loss: 7.7041298393160105
Epoch : 7 loss: 7.6132465079426765
Epoch : 8 loss: 7.325357145629823
Epoch : 9 loss: 6.42864328622818
Epoch : 10 loss: 6.46933015063405
Epoch : 11 loss: 5.507987197488546
Epoch : 12 loss: 6.475949613377452
Epoch : 13 loss: 5.333427561447024
Epoch : 14 loss: 5.587523652240634
Epoch : 15 loss: 5.37697858735919
Epoch : 16 loss: 5.052674955688417
Epoch : 17 loss: 5.522824113257229
Epoch : 18 loss: 5.593496563844383
Epoch : 19 loss: 4.522920983843505
Epoch : 20 loss: 4.589837626554072
Epoch : 21 loss: 5.797779159620404
Epoch : 22 loss: 4.6379057839512825
Epoch : 23 loss: 4.5876549538224936
Epoch : 24 loss: 6.699568632058799
Epoch : 25 loss: 3.9996778052300215
Epoch : 26 loss: 3.347605305723846
Epoch : 27 loss: 4.879490966908634
Epoch : 28 loss: 4.46459306

In [15]:
correct = 0
total = 0
net.eval()
preds = []
with torch.no_grad():
    for data in testdataloader:
        inputs = data
        inputs = torch.tensor(inputs, device=device).float()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
          preds.append(pred.cpu().numpy())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [16]:
preds[:10]

[array(8),
 array(0),
 array(5),
 array(3),
 array(8),
 array(1),
 array(9),
 array(6),
 array(6),
 array(0)]

In [21]:
import numpy as np
preds = np.array(preds)
print(preds.shape)

(18000,)


In [22]:
sample_submission["Category"] = pd.Series(preds)
sample_submission.head()

,Id,Category
0,57808,8
1,4960,0
2,35755,5
3,15543,3
4,48968,8


In [0]:
sample_submission.to_csv("pytorch_submission.csv",index=False)

## kaggle score 0.97907

keras 가 편한 것 같아서 keras로 accuracy 높이려고 노력함...